In [1]:
#加载云盘
from google.colab import drive
drive.mount('/content/drive')
#改变工作目录
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/DF/clound/')
!ls

#过12个小时 空间就会被自动清理
n = len(os.listdir('trainImg/'))
if n > 10000: 
    print("train有图像{}张".format(n))
else:
    os.chdir('/content/drive/My Drive/Colab Notebooks/DF/clound/trainImg')
    !ls
    !unzip -o Train.zip
    os.chdir('/content/drive/My Drive/Colab Notebooks/DF/clound/')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
clound.ipynb	    trainImg	     train_single1.tfrecord
submit_example.csv  Train_label.csv  train_single.tfrecords
train有图像10669张


In [0]:
# os.chdir('/content/drive/My Drive/')
# !ls

In [0]:
# !find -maxdepth 1 -name "*.png" -exec rm -f {} \;
# !ls

In [4]:
import tensorflow as tf
import numpy as np
import keras
from keras import backend as K
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, Concatenate, Add
from keras.layers import BatchNormalization, Activation, Input, SeparableConv2D
from keras.models import Model, Sequential, load_model 
from keras.applications.vgg19 import VGG19, preprocess_input
from keras.utils import plot_model
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import cv2
from PIL import Image
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler

def read_image(name, type):
    """读图片  
    name: 图片名
    type: ‘train' 'test'
    return: Image类型
    """
    if type == 'train':
        path = os.path.join(Train_Img_Dir, name)
    elif type == 'test': 
        path = os.path.join(Test_Img_Dir, name)

    img = Image.open(path) # img.mode = 'RGB'
    if img.mode != 'RGB':
        img = img.convert("RGB") #读取图片的过程中如果遇到非'RGB'就转换格式
    # plt.imshow(img)  
    # plt.show()
    return img 

def get_train_labels():
    """获取所有的train label"""
    labels = pd.read_csv(Train_Label_Path)
    return labels

def get_class_num():
    """获取单标签的分类总数 也就是最后层神经元数目"""
    nls = get_train_labels()
    #获取单标签的训练图片名和label列表
    train_single_labels = nls[nls.apply(lambda x: len(x['Code'])<=2, axis=1)] 
    #单标签 标签类别数目，也是网络最后一层的单元数目
    single_class_num = len(train_single_labels['Code'].unique())
    return single_class_num 

Train_Img_Dir = 'trainImg/'
Train_Label_Path = 'Train_label.csv'
Test_Img_Dir = 'testImg/'
TS_TFRecords_File = 'train_single1.tfrecord' #单标签的图片tfrecoard
TM_TFRecords_File = 'train_multi.tfrecord' #多标签的图片tfrecoard
CLASS_NUM = get_class_num()  #单标签 标签类别数目
# submit_File = 'submit_example.csv'

X_shape = [500, 500, 3] # 预处理后图像的大小
EPOCH = 100 # 训练多少轮
Batch_size = 128  # 训练的batch size

# tf.enable_eager_execution() #调试用


Using TensorFlow backend.


In [0]:
"""****************************************"""
""" 处理数据                 """
"""****************************************"""


class DatasetGenerator:
    def __init__(self):
        self.tls = get_train_labels() #获取训练图片名列表
        #获取单标签的训练图片名和label列表
        self.train_single_labels = self.tls[self.tls.apply(lambda x: len(x['Code'])<=2, axis=1)] 
        self.train_single_num = len(self.train_single_labels) #图片总数
        #获取多标签的训练图片名和label列表(label长度2个以上)
        self.train_multi_labels = self.tls[self.tls.apply(lambda x: len(x['Code'])>2, axis=1)]  
        self.train_multi_num = len(self.train_multi_labels) #图片总数

    def write2TFRecoard(self, type, start=0, end=None, option=None):
        """
        #把图片生成TFRecords文件
        type: TFRecord文件的存放路径 
            'single': 单标签
            'multi'： 多标签
        start, end: 保存图片的
        option: TFRecord文件保存的压缩格式
        """
        #待写 start end参数检查 但是超出index范围没关系 
        if type == 'single': #选择的是单标签的图片
            tfr_file = TS_TFRecords_File
            nls = self.train_single_labels
        else: # type == 'multi'
            tfr_file = TM_TFRecords_File
            nls = self.train_multi_labels

        trf_writer = tf.python_io.TFRecordWriter(tfr_file, options=option)

        nls = nls.sample(frac=1).reset_index(drop=True)    #打乱顺序 frac是要返回的比例 1=100%
        for index, (name, label) in nls[start:end].iterrows():
            img = read_image(name, 'train')
            img = self.__preprocessing(img) #裁剪 归一化等预处理
            img_shape = img.shape
            img = img.reshape(-1) #变成一维
            img_list = img.tolist() 
            #多标签处理
            #if len(label)
            feature_internal = {
                    'image_raw' : tf.train.Feature(float_list = tf.train.FloatList(value=img_list)), #内层feature编码方式
                    'img_shape' : tf.train.Feature(int64_list = tf.train.Int64List(value=img_shape)),
                    'label'   : tf.train.Feature(int64_list = tf.train.Int64List(value=[int(label)]))     
                    }
            #使用tf.train.Example将features编码数据封装成特定的PB协议格式
            example = tf.train.Example(features=tf.train.Features(feature=feature_internal))
            #将序列化为字符串的example数据写入协议缓冲区
            print('No.{}: trf_writer写入{} label:{}'.format(index, name, label))
            trf_writer.write(example.SerializeToString())
        #关闭TFRecords文件操作接口    
        trf_writer.close()

    def get_dataset_from_TFRecoard(self, type, batch_size=None):
        """
        ##从tfr_files指定的TFRecords文件，初始化一个dataset
        :param 
            type: TFRecord文件的存放路径 
                'single': 单标签
                'multi'： 多标签            
            batch_size: 参数表示batch的消息 None不处理 
        :return:
            可迭代的dataset 
        ！可以考虑用slim模块实现
        """
        if type == 'single': #选择的是单标签的图片
            tfr_files = TS_TFRecords_File
        else: # type == 'multi'
            tfr_files = TM_TFRecords_File

        # 定义TFRecordDataset
        dataset_raw = tf.data.TFRecordDataset(tfr_files)
        # for raw_record in raw_dataset.take(10): print(repr(raw_record)) #测试用
        # Set the number of datapoints you want to load and shuffle 
        dataset = dataset_raw.shuffle(buffer_size=1000)
        #执行解析函数 得到数据集    
        dataset = dataset.map(self.__parse_function)
        # for parsed_record in dataset.take(10): print(repr(parsed_record)) #测试用
        # 定义batchsize大小
        if batch_size:
            dataset = dataset.batch(batch_size)
        # 不加参数->无限重复数据集
        dataset = dataset.repeat()
        return dataset

    def get_train_data_from_TFRecoard(self, batch_size=None):
        """
        ##从tfr_files指定的TFRecords文件，初始化一个dataset,并读取train数据
        :param 
            batch_size: 参数表示batch的消息 None不处理 
        :return: 
            可迭代的image label数据
        """
        dataset = self.get_dataset_from_TFRecoard(type='single', batch_size=batch_size)        
        # Create an iterator
        iterator = dataset.make_one_shot_iterator()
        
        # Create your tf representation of the iterator
        image, label = iterator.get_next()
        print('reshape.........,', [batch_size]+X_shape)
        image = tf.reshape(image, [batch_size]+X_shape)
        # label = tf.reshape(label, [batch_size])
        label = tf.one_hot(label, CLASS_NUM)
        return image, label

    def __preprocessing(self, img):
        """对图像数据进行预处理"""
        #其他处理。。。
        #resize crop变成H*W大小
        img = self.__resize(img)

        # img_norm = tf.image.per_image_standardization(timg) #tensorflow中对图像标准化预处理的API
        img_array = np.asarray(img, np.float32)

        img_norm = img_array/255.  #[0,1]归一化
        return img_norm

    def __resize(self, img):
        """对图像裁切 缩减"""
        H, W = X_shape[:2]   #输入模型需要的图片的高 宽
        h, w = img.size[::-1]  #原始图片的高 宽
        if h > w:  #如果原图宽比较小
            if w > W*1.1:  #宽比要求的尺寸高10%以上
                image = img.resize((int(W*1.1), int(h/w*W*1.1)), Image.ANTIALIAS)  #宽缩到110% 剩下的裁剪掉
            elif w < W:  #宽比要求的尺寸小
                image = img.resize((W, int(h*W/w)), Image.ANTIALIAS) #宽放大到需要尺寸 高同步放大
            else: image = img
        else: #原图高比较小
            if h > H*1.15:  #高 多15%以上
                image = img.resize((int(w/h*H*1.15), int(H*1.15)), Image.ANTIALIAS)
            elif h < H:
                image = img.resize((int(w*H/h), H), Image.ANTIALIAS)
            else: image = img
        #对resize好的图片 进行裁剪，满足尺寸H,W
        h, w = image.size[::-1] #resize后的尺寸
        top = int(0.2 * (h-H)) #上高裁切 取 多余的20%
        left = int(0.5 * (w-W)) #宽度多余的 左右各裁剪一半
        image = image.crop((left, top, left+W, top+H)) #四周裁剪 留下H*W区域
        return image

    def __parse_function(self, example_proto):
        """解析函数 
        :param example_proto: example序列化后的样本tf_serialized
        """
        features = {
            'image_raw' : tf.VarLenFeature(dtype=tf.float32),
            'img_shape' : tf.FixedLenFeature(shape=(3,), dtype=tf.int64),
            'label' : tf.FixedLenFeature(shape=(), dtype=tf.int64)
            }
        # 把序列化样本和解析字典送入函数里得到解析的样本
        parsed_example = tf.parse_single_example(example_proto, features) #返回字典
        # 解码 
        img = tf.sparse_tensor_to_dense(parsed_example['image_raw'], default_value=0) # 稀疏->密集表示
        img = tf.reshape(img, parsed_example['img_shape']) # 转换tensor形状
        # 如果使用dataset作为keras中，model.fit函数等的参数，则需要使用one_hot编码
        # 在tensorflow中，基本是不需要的，可以直接返回example['label']。
        # label = tf.one_hot(parsed_example['label'], self.train_single_num)
        # label = tf.cast(parsed_example['label'], tf.uint8)
        label = parsed_example['label']
        return img, label


    def _get_train_img(self, start=0, end=100):
        """测试用 读图片
        return: Image格式
        """
        for i in range(start, end):
            (name, label) = self.train_single_labels.iloc[i]        
            img = read_image(name, 'train')
            yield(img, label)



In [6]:
dg = DatasetGenerator()
#判断文件sfou是否存在（colab会自动删除）
if not os.path.exists(TS_TFRecords_File): 
    dg.write2TFRecoard('single', 0, 500) #写tfrecoard

#Get datatensors 返回一个batch的数据: [batch,x,x,3] 和 one_hot label 
imgs, labels = dg.get_train_data_from_TFRecoard(batch_size=10)


Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.
reshape........., [10, 500, 500, 3]


In [7]:
"""****************************************"""
""" 模型                 """
"""****************************************"""

def Conv2D_BN(x, filters, kernel_size, padding='same', activation='relu'):
    """加BN层的卷积
    # Returns
        Output tensor after applying `Conv2D` and `BatchNormalization`
    """
    x = Conv2D(filters, kernel_size, padding=padding)(x)
    x = BatchNormalization()(x)
    if activation: 
        x = Activation(activation)(x)
    return x

def inception_module(x, params, concat_axis):
    """
    inception块
    param :
        x: 输入的张量，维度为（m, n_H, n_W, n_C）
        params: 每层输出通道数
    """
    (branch1, branch2, branch3, branch4) = params #每层输出通道数
    #inception module
    pathway1 = Conv2D_BN(x, branch1, (1,1))
    pathway2 = Conv2D_BN(x, branch2[0], (1,1))
    pathway2 = Conv2D_BN(pathway2, branch2[1], (3,3))
    pathway3 = Conv2D_BN(x, branch3[0], (1,1))
    pathway3 = Conv2D_BN(pathway3, branch3[1], (3,3))
    pathway3 = Conv2D_BN(pathway3, branch3[1], (3,3))
    pathway4 = MaxPool2D(pool_size=(3,3), strides=1, padding='same')(x)
    pathway4 = Conv2D_BN(pathway4, branch4, (1,1))
    return Concatenate(axis=concat_axis)([pathway1,pathway2,pathway3,pathway4])

def resdual_module(inputs, params, name):
    """
    三层的恒等残差块
    param :
        inputs: 输入的张量，维度为（m, n_H, n_W, n_C）
        params: 每层输出通道数

    """
    with tf.variable_scope(name):
        (branch1, branch2, branch3) = params #每层输出通道数
        x = Conv2D_BN(inputs, branch1, kernel_size=(1,1), padding='valid', activation='relu')
        x = Conv2D_BN(x, branch2, kernel_size=(3,3), padding='same', activation='relu')
        x = Conv2D_BN(x, branch3, kernel_size=(1,1), padding='valid', activation=None)
        x = Add()([x, inputs]) 
        x = Activation('relu')(x)
    return x

model_input = Input(tensor=imgs) #迭代的输入图片
#load VGG19 模型，不包括全连接层
base_model = VGG19(weights = 'imagenet', 
           include_top = False, 
           input_tensor = model_input
           )
x = Flatten()(base_model.output)
x = Dense(128)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.5)(x)
pridictions = Dense(CLASS_NUM, activation='softmax')(x)
# 构建我们需要训练的完整模型
model = Model(base_model.inputs, pridictions)
#冻结
for layer in base_model.layers:  
    layer.trainable = False 

# 编译模型（一定要在锁层以后操作）
model.compile(optimizer='adam',
       loss='categorical_crossentropy',
       metrics=['acc'],
       target_tensors=[labels] #迭代的label
       )

early_stopping = keras.callbacks.EarlyStopping(monitor='loss', patience=5)
cp_path = 'weights-{epoch:02d}-{val_acc:.3f}.hdf5' #checkpoint保存的路径
checkpoint = keras.callbacks.ModelCheckpoint(filepath=cp_path,
                        verbose=1,
                        save_best_only=True,
                        mode='max'
                        )

# x = Conv2D_BN(inputs, 64, (3,3), padding='same')
# x = MaxPool2D()(x)
 
# x = resdual_module(x, (16,16,64), name="Res_block1")
# x = resdual_module(x, (16,16,64), name="Res_block2")
# x = resdual_module(x, (16,16,64), name="Res_block3")
# x = resdual_module(x, (16,16,64), name="Res_block3")

# x = inception_module(x, params=(32,(32,64),(24,48),16), concat_axis=-1)
# x = MaxPool2D()(x)








Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



In [0]:
# model.summary()

In [9]:
hist = model.fit(epochs=10,
        verbose=2,
        steps_per_epoch=20,
        callbacks=[checkpoint], #early_stopping, 
        )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/10
 - 22s - loss: 3.5629 - acc: 0.1000
Epoch 2/10
 - 16s - loss: 2.6245 - acc: 0.3250
Epoch 3/10
 - 16s - loss: 2.1408 - acc: 0.4450
Epoch 4/10
 - 16s - loss: 1.7272 - acc: 0.5300
Epoch 5/10
 - 16s - loss: 1.5021 - acc: 0.5650
Epoch 6/10
 - 16s - loss: 1.2811 - acc: 0.6650
Epoch 7/10
 - 16s - loss: 1.2416 - acc: 0.6500
Epoch 8/10
 - 16s - loss: 1.1246 - acc: 0.7050
Epoch 9/10
 - 16s - loss: 0.9637 - acc: 0.7450
Epoch 10/10
 - 16s - loss: 0.9212 - acc: 0.7800


In [0]:
# dataset = dg.get_dataset_from_TFRecoard(type='single', batch_size=1)

In [0]:
# hist = model.fit(img, label, epochs=2, verbose=2, steps_per_epoch=1000)

In [0]:
# model.evaluate_generator(img,)
# model.predict_generator(img, steps=100)

In [0]:
###### 以下测试 /////////////////////////////////////////////
sess = tf.InteractiveSession() 
sess.run(tf.global_variables_initializer())

In [15]:
dg.write2TFRecoard('single', 0, 3000) #写入到tfrecoard

No.0: trf_writer写入b64ed60f63a1400daa1d944515aa500c.png label:8
No.1: trf_writer写入70277d501bf640769c0f7d89bdec1d8d.jpg label:10
No.2: trf_writer写入419e2cb7e7c64659a9d035b228349f9b.jpg label:18
No.3: trf_writer写入2a069c8f1c4a4564bd0f53368a27fbbf.jpg label:10
No.4: trf_writer写入23b898556e964e13954654a24f83e9f2.jpg label:28
No.5: trf_writer写入0971e56874bd42d088d840ab1c7f7475.png label:21
No.6: trf_writer写入515ae6337c494936907c63e3536ef9ae.png label:8
No.7: trf_writer写入92430aa3f6ef49cdaea978a85edf3418.jpg label:28
No.8: trf_writer写入c0c4231f27bf4c2e8b3fdd6221c57e7a.png label:16
No.9: trf_writer写入e460745077ee4c2399d700de06b9ddbe.jpg label:25
No.10: trf_writer写入e0d9c54e86bb4b1bb9d2f42a0230421d.jpg label:27
No.11: trf_writer写入1546a478353b4dcf8959d4671e298bc0.jpg label:2
No.12: trf_writer写入af7ce013aaac4a348e4a2f90ef53900b.jpg label:18
No.13: trf_writer写入b41f03ebd6d94878b5f370aa1937f975.jpg label:10
No.14: trf_writer写入99752a44210346f79c9bb73baef1eb3f.jpg label:2
No.15: trf_writer写入218ac852a62845f2996a

In [0]:
#ok 测试读取tfrecoard数据ky
imgs, labels = dg.get_train_data_from_TFRecoard(batch_size=5)
for i in range(3):
    image = imgs.eval()
    label = labels.eval()
    print('batch {} ... shape:{} '.format(i+1, image.shape))
    for j in range(image.shape[0]):
        print('No.{} image label:{}'.format(j+1, label[j]))
        plt.imshow(image[j])
        plt.show()

In [0]:
i = 1
for timg, l in dg._get_train_img(): #取测试图片
    i += 1
#     print(l)
#     plt.imshow(timg), timg.size
#     plt.show()    
#     r = dg._resize(timg)
#     plt.imshow(r), r.size 
#     plt.show()
    if i > 5: break

In [0]:
i = np.array([5])

In [0]:
i = (np.reshape(i, [1,]), 29)

In [0]:
i.eval()